In [144]:
import pandas as pd

# Load the datasets
customers_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagements_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [145]:
## DATA CLEANING

In [146]:
# Convert date columns to datetime
customers_df['join_date'] = pd.to_datetime(customers_df['join_date'])
customers_df['last_purchase_date'] = pd.to_datetime(customers_df['last_purchase_date'])
marketing_df['campaign_date'] = pd.to_datetime(marketing_df['campaign_date'])
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])

In [147]:
# Define the cutoff date
cutoff_date = pd.to_datetime('2024-07-17')

# Convert dates later than 7/17/2024 to 7/17/2024
customers_df['join_date'] = customers_df['join_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
customers_df['last_purchase_date'] = customers_df['last_purchase_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
marketing_df['campaign_date'] = marketing_df['campaign_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
transactions_df['transaction_date'] = transactions_df['transaction_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)

In [148]:
# Aggregate marketing responses
marketing_agg = marketing_df.groupby('customer_id')['response'].value_counts().unstack(fill_value=0).reset_index()
marketing_agg.columns = ['customer_id', 'no_response', 'yes_response']

# Aggregate transaction data
transactions_agg = transactions_df.groupby('customer_id').agg(
    total_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='sum'),
    number_of_transactions=pd.NamedAgg(column='transaction_id', aggfunc='count')
).reset_index()

# Merge all dataframes
merged_df = customers_df.merge(engagements_df, on='customer_id', how='left') \
                        .merge(marketing_agg, on='customer_id', how='left') \
                        .merge(transactions_agg, on='customer_id', how='left')

# Fill NaN values for no_response and yes_response with 0
merged_df['no_response'] = merged_df['no_response'].fillna(0)
merged_df['yes_response'] = merged_df['yes_response'].fillna(0)

# Fill NaN values for total_transaction_amount and number_of_transactions with 0
merged_df['total_transaction_amount'] = merged_df['total_transaction_amount'].fillna(0)
merged_df['number_of_transactions'] = merged_df['number_of_transactions'].fillna(0)

In [149]:
# Impute 'Unknown' for missing gender values
merged_df['gender'] = merged_df['gender'].fillna('Unknown')

In [150]:
# Impute missing age with KNN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Convert date columns to numeric
merged_df['join_date_numeric'] = merged_df['join_date'].astype('int64') // 10**9
merged_df['last_purchase_date_numeric'] = merged_df['last_purchase_date'].astype('int64') // 10**9

# Encode categorical variables
le_gender = LabelEncoder()
merged_df['gender_encoded'] = le_gender.fit_transform(merged_df['gender'])

le_location = LabelEncoder()
merged_df['location_encoded'] = le_location.fit_transform(merged_df['location'])

# Prepare data for KNN imputation
impute_cols = ['age', 'join_date_numeric', 'last_purchase_date_numeric', 'gender_encoded', 'location_encoded']
impute_data = merged_df[impute_cols]

# Perform KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)
imputed_data = knn_imputer.fit_transform(impute_data)

# Assign imputed age values back to the dataframe
merged_df['age'] = imputed_data[:, 0]

# Drop intermediate columns
merged_df = merged_df.drop(columns=['join_date_numeric', 'last_purchase_date_numeric', 'gender_encoded', 'location_encoded'])

In [151]:
## Feature Engineering

In [152]:
# Calculate Recency, Frequency, and Monetary Value (RFM) features
import numpy as np

# Define the cutoff date
cutoff_date = pd.to_datetime('2024-07-17')

# Calculate the joined month (days difference and convert to months)
merged_df['joined_days'] = (cutoff_date - merged_df['join_date']).dt.days
merged_df['joined_month'] = (merged_df['joined_days'] / 30.44).astype(int)
merged_df['joined_month'] = merged_df['joined_month'].replace(0, np.nan)

# Calculate recency in days and convert to months
merged_df['recency_days'] = (cutoff_date - merged_df['last_purchase_date']).dt.days
merged_df['recency'] = (merged_df['recency_days'] / 30.44).astype(int)

# Calculate frequency (number of transactions per month)
merged_df['frequency'] = merged_df['number_of_transactions'] / merged_df['joined_month']

# Calculate monetary value (transaction value per month)
merged_df['clv_monthly'] = merged_df['total_transaction_amount'] / merged_df['joined_month']

# Fill NaN values for joined_month with a placeholder
merged_df['joined_month'] = merged_df['joined_month'].fillna('.')

# Drop intermediate columns
merged_df = merged_df.drop(columns=['joined_days', 'recency_days'])

In [153]:
# Calculate the number of 'yes_response' per transaction
merged_df['yes_response_per_transaction'] = merged_df['yes_response'] / merged_df['number_of_transactions']

# Calculate the number of 'no_response' per transaction
merged_df['no_response_per_transaction'] = merged_df['no_response'] / merged_df['number_of_transactions']

# Calculate the number of site visits per transaction
merged_df['site_visits_per_transaction'] = merged_df['number_of_site_visits'] / merged_df['number_of_transactions']

# Calculate the number of emails opened per transaction
merged_df['emails_opened_per_transaction'] = merged_df['number_of_emails_opened'] / merged_df['number_of_transactions']

# Calculate the number of clicks per transaction
merged_df['clicks_per_transaction'] = merged_df['number_of_clicks'] / merged_df['number_of_transactions']

In [154]:
# use the 80-20 rule, sorting the customer with clv_monthly
# labelling the highest clv_monthly customers who cumulatively contribute 80% of the total monthly clv as 'High-Value'
# other as ‘Low-Value’

# Calculate the cumulative contribution of each customer to the total monthly CLV
merged_df = merged_df.sort_values(by='clv_monthly', ascending=False)
merged_df['cumulative_clv'] = merged_df['clv_monthly'].cumsum()
total_clv = merged_df['clv_monthly'].sum()
merged_df['cumulative_clv_percentage'] = merged_df['cumulative_clv'] / total_clv

# Label customers based on their contribution to the total monthly CLV
merged_df['value_label'] = np.where(merged_df['cumulative_clv_percentage'] <= 0.80, 'High-Value', 'Low-Value')

# Drop intermediate columns
merged_df = merged_df.drop(columns=['cumulative_clv', 'cumulative_clv_percentage'])

In [155]:
# Create a copy of merged_df as eda_df
eda_df = merged_df.copy()

In [156]:
## EDA

In [158]:
# Check the imbalance
# Calculate the distribution of the target variable
class_distribution = eda_df['value_label'].value_counts(normalize=True)

# Print the class distribution
print(class_distribution)

value_label
Low-Value     0.6493
High-Value    0.3507
Name: proportion, dtype: float64


In [159]:
## Model Selection

In [160]:
# split data into X variables and y variable
# Define the y variable
y = merged_df['value_label']

# Define the X variables
X = merged_df[['age', 'gender', 'recency', 'frequency', 'yes_response_per_transaction', 
               'no_response_per_transaction', 'site_visits_per_transaction', 
               'emails_opened_per_transaction', 'clicks_per_transaction']]

In [161]:
# scale the features using MinMaxScaler
# encode the categorical variables using OneHotEncoder
# split the data into training and testing sets
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Apply MinMaxScaler to X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[['age', 'recency', 'frequency', 'yes_response_per_transaction',
                                   'no_response_per_transaction', 'site_visits_per_transaction',
                                   'emails_opened_per_transaction', 'clicks_per_transaction']])

# Encode the gender column in X
encoder = OneHotEncoder(drop='first')
gender_encoded = encoder.fit_transform(X[['gender']]).toarray()

# Concatenate scaled numerical features and encoded categorical features
X_encoded = np.hstack((X_scaled, gender_encoded))

# Encode the y variable
y_encoded = encoder.fit_transform(y.to_frame()).toarray()

# Split the data into training and testing sets, reserving 30% for testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.30, random_state=42)


In [162]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# Define numeric and categorical columns
numeric_columns = ['age', 'recency', 'frequency', 'yes_response_per_transaction', 
                   'no_response_per_transaction', 'site_visits_per_transaction', 
                   'emails_opened_per_transaction', 'clicks_per_transaction']
categorical_columns = ['gender']

# Create a pre-processing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# Define the models
knn = KNeighborsClassifier()
logreg = LogisticRegression()
nb = GaussianNB()

# Split the data into training and testing sets, reserving 30% for testing
X = merged_df[numeric_columns + categorical_columns]
y = merged_df['value_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Create pipelines for each model
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', knn)])
logreg_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', logreg)])
nb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', nb)])

# Evaluate the models using cross-validation
knn_scores = cross_val_score(knn_pipeline, X_train, y_train, cv=5)
logreg_scores = cross_val_score(logreg_pipeline, X_train, y_train, cv=5)
nb_scores = cross_val_score(nb_pipeline, X_train, y_train, cv=5)

# Print the cross-validation scores
print(np.mean(knn_scores))
print(np.mean(logreg_scores))
print(np.mean(nb_scores))

0.8630000000000001
0.8752857142857143
0.8022857142857143


In [167]:
# Improved KNN Model
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for KNN
param_grid = {
    'classifier__n_neighbors': [5, 7, 9, 11, 13, 15, 35, 45, 55],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Create the pipeline for KNN
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())])

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train,y_train)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score}')

Best Parameters: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 11, 'classifier__weights': 'distance'}
Best Score: 0.8902857142857142
